## Import All the Library

In [1]:
import os 
import io
import matplotlib.pyplot as plt
from google.cloud import vision
from google.cloud.vision import enums
import cv2 as cv

from PIL import Image
import urllib.request

import gzip

from google.cloud.vision_v1.proto import image_annotator_pb2
from google.protobuf.json_format import MessageToDict

import re
from google.cloud import vision
from google.cloud import storage
from google.protobuf import json_format
from google.cloud import storage

from google.cloud import language_v1
from google.cloud.language_v1 import enums

import argparse
import io
import json
import os

from google.cloud import language
import numpy
import six
import pandas as pd

## Set the Google application Credential

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="my-project-12195-46fed19566f2.json"

## 1-Parsing documents containing forms 

Document AI can detect and parse text from PDF, TIFF, GIF files stored in Cloud Storage, including text that contains unstructured data in the form documents.

## 1.1-Small file online processing pdf

** STEP**

1-Enable document AI in google cloud

2-Create a function that will take pdf path(containg any application form) as an input and will return all the relevent information from that pdf.

3-Project_id which is the id of the project under which API is anabled  is also passed inside the function

4-Use document undertanding service client which is used to parse structured information from unstructured or
semi-structured documents using state-of-the-art Google AI such as natural language, computer vision, and translation.

5-The mimi_type or type of document is applicatiion/pdf

6- Improve form parsing results by providing key-value pair hints.For each key hint, key is text that is likely to appear in the document as a form field name (i.e. "DOB").Value types are optional, but can be one or more of:ADDRESS, LOCATION, ORGANIZATION, PERSON, PHONE_NUMBER, ID,NUMBER, EMAIL, PRICE, TERMS, DATE, NAME

7-Use ProcessDocumentRequest request which will process one document at a time

8-Return the response

#### Create the function parse_form

In [3]:
from google.cloud import documentai_v1beta2 as documentai


def parse_form(project_id,input_uri):
    """Parse a form"""

    client = documentai.DocumentUnderstandingServiceClient()

    gcs_source = documentai.types.GcsSource(uri=input_uri)

    # mime_type can be application/pdf, image/tiff,
    # and image/gif, or application/json
    input_config = documentai.types.InputConfig(
        gcs_source=gcs_source, mime_type='application/pdf')

    key_value_pair_hints = [
        documentai.types.KeyValuePairHint(key='Emergency Contact',
                                          value_types=['NAME']),
        documentai.types.KeyValuePairHint(
            key='Referred By')
    ]

    # Setting enabled=True enables form extraction
    form_extraction_params = documentai.types.FormExtractionParams(
        enabled=True, key_value_pair_hints=key_value_pair_hints)

    # Location can be 'us' or 'eu'
    parent = 'projects/{}/locations/us'.format(project_id)
    request = documentai.types.ProcessDocumentRequest(
        parent=parent,
        input_config=input_config,
        form_extraction_params=form_extraction_params)

    document = client.process_document(request=request)
    return document

#### Call the parse_form function with project id and uri path where pdf is stored
#### Print the document

In [4]:
project_id ='my-project-12195'
input_uri ='gs://training-12195/form.pdf'
document=parse_form(project_id,input_uri)
print(document)

mime_type: "application/pdf"
text: "FakeDoc M.D.\nHEALTH INTAKE FORM\nPlease fill out the questionnaire carefully. The information you provide will be used to complete\nyour health profile and will be kept confidential.\nDate:\n9/14/19\nName: Sally Walker\nDOB: 09/04/1986\nAddress: 24 Barney Lane City: Tonaco State: NJ Zip: 07082\nEmail: Sally, waller@cmail.com_Phone #:_(906) 917-3486\nGender: _F\nSingle Occupation: Software Engineer\nReferred By: None\nEmergency Contact: Eva Walker Emergency Contact Phone: (906)334-89766\nMarital Status:\nDescribe your medical concerns (symptoms, diagnoses, etc):\nRanny nose, mucas in thoat, weakness,\naches, chills, tired\nAre you currently taking any medication? (If yes, please describe):\nVyvanse (25mg) daily for attention\n"
pages {
  page_number: 1
  dimension {
    width: 418.0
    height: 492.0
    unit: "points"
  }
  layout {
    text_anchor {
      text_segments {
        end_index: 716
      }
    }
    bounding_poly {
      vertices {
    

The document (response) contain all the relecvent detail from the form .To get the relevent information we need to iterate over the reponse .

#### Lets extract relevant information like field name ,field value confidence score from the response

**STEP**

1- From the document extract document.pages 

2- from pages extract form field that contain all the detail about the form

3-to get the value of form field and form value ,Create a function that will take field name and field value param as input and return all the value from it

4- Inside the function use el.text_anchor.text_segments to get text from the document

In [5]:
def _get_text(el):
        """Doc AI identifies form fields by their offsets
        in document text. This function converts offsets
        to text snippets.
        """
        response = ''
        # If a text segment spans several lines, it will
        # be stored in different text segments.
        for segment in el.text_anchor.text_segments:
            start_index = segment.start_index
            end_index = segment.end_index
            response += document.text[start_index:end_index]
        return response

for page in document.pages:
    print('Page number: {}'.format(page.page_number))
    for form_field in page.form_fields:
        print('Field Name: {}\tConfidence: {}'.format(_get_text(form_field.field_name),form_field.field_name.confidence))
        print('Field Value: {}\tConfidence: {}'.format(_get_text(form_field.field_value),form_field.field_value.confidence))

Page number: 1
Field Name: #:	Confidence: 0.9999823570251465
Field Value: _(906) 917-3486
	Confidence: 0.9999823570251465
Field Name: Emergency Contact: 	Confidence: 0.9999718070030212
Field Value: Eva Walker 	Confidence: 0.9999718070030212
Field Name: Marital Status:
	Confidence: 0.9999508261680603
Field Value: Single 	Confidence: 0.9999508261680603
Field Name: Gender: 	Confidence: 0.9999330639839172
Field Value: _F
	Confidence: 0.9999330639839172
Field Name: Occupation: 	Confidence: 0.9999136328697205
Field Value: Software Engineer
	Confidence: 0.9999136328697205
Field Name: Referred By: 	Confidence: 0.9998615980148315
Field Value: None
	Confidence: 0.9998615980148315
Field Name: Date:
	Confidence: 0.9998577833175659
Field Value: 9/14/19
	Confidence: 0.9998577833175659
Field Name: DOB: 	Confidence: 0.9997154474258423
Field Value: 09/04/1986
	Confidence: 0.9997154474258423
Field Name: Address: 	Confidence: 0.999135434627533
Field Value: 24 Barney Lane 	Confidence: 0.999135434627533
Fi

**Result**

1-For each of the text we have three value field name,field valu and confidence score


## 1.2-Large file offline processing
Asynchronous ("offline") requests targets longer documents and allows you to set the number of pages in the output files. This request starts a long-running operation. When this operation finishes it stores output as a JSON file in a specified Cloud Storage bucket.

#### The step are same as above function.Only difference is to pass destination uri where the extracted data in json format is stored

#### Create a batch_parse_form that take project_id,input_uri,destination_uri and store the extracted in json format

In [6]:
def batch_parse_form(project_id,input_uri,destination_uri):
    """Parse a form"""

    client = documentai.DocumentUnderstandingServiceClient()

    gcs_source = documentai.types.GcsSource(uri=input_uri)

    # mime_type can be application/pdf, image/tiff,
    # and image/gif, or application/json
    input_config = documentai.types.InputConfig(
        gcs_source=gcs_source, mime_type='application/pdf')

    # where to write results
    output_config = documentai.types.OutputConfig(
        gcs_destination=documentai.types.GcsDestination(
            uri=destination_uri),
        pages_per_shard=1  # Map one doc page to one output page
    )

    # Improve form parsing results by providing key-value pair hints.
    # For each key hint, key is text that is likely to appear in the
    # document as a form field name (i.e. "DOB").
    # Value types are optional, but can be one or more of:
    # ADDRESS, LOCATION, ORGANIZATION, PERSON, PHONE_NUMBER, ID,
    # NUMBER, EMAIL, PRICE, TERMS, DATE, NAME
    key_value_pair_hints = [
        documentai.types.KeyValuePairHint(
            key='Emergency Contact',
            value_types=['NAME']),
        documentai.types.KeyValuePairHint(
            key='Referred By')
    ]

    # Setting enabled=True enables form extraction
    form_extraction_params = documentai.types.FormExtractionParams(
        enabled=True, key_value_pair_hints=key_value_pair_hints)

    # Location can be 'us' or 'eu'
    parent = 'projects/{}/locations/us'.format(project_id)
    request = documentai.types.ProcessDocumentRequest(
        input_config=input_config,
        output_config=output_config,
        form_extraction_params=form_extraction_params)

    # Add each ProcessDocumentRequest to the batch request
    requests = []
    requests.append(request)

    batch_request = documentai.types.BatchProcessDocumentsRequest(
        parent=parent, requests=requests
    )

    operation = client.batch_process_documents(batch_request)

    # Wait for the operation to finish
    operation.result()

    # Results are written to GCS. Use a regex to find
    # output files
    match = re.match(r'gs://([^/]+)/(.+)', destination_uri)
    output_bucket = match.group(1)
    prefix = match.group(2)

    storage_client = storage.client.Client()
    bucket = storage_client.get_bucket(output_bucket)
    blob_list = list(bucket.list_blobs(prefix=prefix))
    print('Output files:')
    for blob in blob_list:
        print(blob.name)


#### Call the function qnd pass all three parameter .

In [7]:
project_id ='my-project-12195'
input_uri ='gs://training-12195/form.pdf'
destination_uri ='gs://training-12195/documentai/'
batch_parse_form(project_id,input_uri,destination_uri)

Output files:
documentai/
documentai/output-page-1-to-1.json


The output will be stored in the google storage link

## 2-Parsing documents containing tables as PDF

**STEP**

Step are similar to parsing the form from pdf .The only difference is use **documentai.types.TableExtractionParams** to extract table

In [8]:
def parse_table(project_id,input_uri):
    """Parse a form"""

    client = documentai.DocumentUnderstandingServiceClient()

    gcs_source = documentai.types.GcsSource(uri=input_uri)

    # mime_type can be application/pdf, image/tiff,
    # and image/gif, or application/json
    input_config = documentai.types.InputConfig(
        gcs_source=gcs_source, mime_type='application/pdf')

    # Improve table parsing results by providing bounding boxes
    # specifying where the box appears in the document (optional)
    table_bound_hints = [
        documentai.types.TableBoundHint(
            page_number=1,
            bounding_box=documentai.types.BoundingPoly(
                # Define a polygon around tables to detect
                # Each vertice coordinate must be a number between 0 and 1
                normalized_vertices=[
                    # Top left
                    documentai.types.geometry.NormalizedVertex(
                        x=0,
                        y=0
                    ),
                    # Top right
                    documentai.types.geometry.NormalizedVertex(
                        x=1,
                        y=0
                    ),
                    # Bottom right
                    documentai.types.geometry.NormalizedVertex(
                        x=1,
                        y=1
                    ),
                    # Bottom left
                    documentai.types.geometry.NormalizedVertex(
                        x=0,
                        y=1
                    )
                ]
            )
        )
    ]

    # Setting enabled=True enables form extraction
    table_extraction_params = documentai.types.TableExtractionParams(
        enabled=True, table_bound_hints=table_bound_hints)

    # Location can be 'us' or 'eu'
    parent = 'projects/{}/locations/us'.format(project_id)
    request = documentai.types.ProcessDocumentRequest(
        parent=parent,
        input_config=input_config,
        table_extraction_params=table_extraction_params)

    document = client.process_document(request=request)
    return document   

#### Call the function and pass project id and uri and print the response

In [9]:
project_id ='my-project-12195'
input_uri ='gs://training-12195/invoice.pdf'
document=parse_table(project_id,input_uri)
print(document)

mime_type: "application/pdf"
text: "TERMS: 6 month contract\nDUE: 01/01/2025\nNOTES:\nFROM: Company ABC\nuser@companyabc.com\nADDRESS: 111 Main Street\nAnytown, USA\nItem Description Quantity Price Amount\nTool A 500 $1.00 $500.00\nService B 1 $900.00 $900.00\nResource C 50 $12.00 $600.00\nSupplies used for Project Q.\nTO: John Doe\njohndoe@email.com\nADDRESS: 222 Main Street\nAnytown, USA\nSubtotal $2000.00\nTax $140.00\nBALANCE DUE $2140.00\nDATE: 01/01/1970\nINVOICE: NO. 001\nInvoice\n"
pages {
  page_number: 1
  dimension {
    width: 612.0
    height: 792.0
    unit: "points"
  }
  layout {
    text_anchor {
      text_segments {
        end_index: 435
      }
    }
    confidence: 1.0
    bounding_poly {
      vertices {
      }
      vertices {
        x: 612
      }
      vertices {
        x: 612
        y: 792
      }
      vertices {
        y: 792
      }
      normalized_vertices {
      }
      normalized_vertices {
        x: 1.0
      }
      normalized_vertices {
     

#### Lets Extract the relevent information from the text

**Step**

1-To create a dataframe firstly we will create a blank dataframe

2-To get the column name iterate over table.header_rows and call the function tht will extract the text .

3-convert the string value into list and sotred it

4- Repeat the same step to get the rows value of dataframe and use dataframe.append to store each row

5-dataframe.columns to get column name

6- print the dataframe

In [17]:
def _get_text(el):
        """Convert text offset indexes into text snippets.
        """
        response = ''
        # If a text segment spans several lines, it will
        # be stored in different text segments.
        for segment in el.text_anchor.text_segments:
            start_index = segment.start_index
            end_index = segment.end_index
            response += document.text[start_index:end_index]
        return response

for page in document.pages:
    print('Page number: {}'.format(page.page_number))
    for table_num, table in enumerate(page.tables):
        print('Table {}: '.format(table_num))
        for row_num, row in enumerate(table.header_rows):
            cells = '\t'.join([_get_text(cell.layout) for cell in row.cells])
            a=cells.split('\t')
            a=list(map(lambda x:x.strip(),a))
            print('Header Row {}: {}'.format(row_num, cells))
            form=pd.DataFrame()
            #print(form)
        for row_num, row in enumerate(table.body_rows):
            cells = '\t'.join([_get_text(cell.layout) for cell in row.cells])
            b=cells.split('\t')
            b=list(map(lambda x:x.strip(),b))
            form=form.append([b],ignore_index=True)
            print('Row {}: {}'.format(row_num, cells))
form.columns=a

Page number: 1
Table 0: 
Header Row 0: Item Description 	Quantity 	Price 	Amount

Row 0: Tool A 	500 	$1.00 	$500.00

Row 1: Service B 	1 	$900.00 	$900.00

Row 2: Resource C 	50 	$12.00 	$600.00

Row 3: 		Subtotal 	$2000.00

Row 4: 		Tax 	$140.00

Row 5: 		BALANCE DUE 	$2140.00



#### Dataframe is created with name form Lets print the dataframe

In [12]:
form.to_csv('table.csv')
form

,Item Description,Quantity,Price,Amount
0,Tool A,500,$1.00,$500.00
1,Service B,1,$900.00,$900.00
2,Resource C,50,$12.00,$600.00
3,,,Subtotal,$2000.00
4,,,Tax,$140.00
5,,,BALANCE DUE,$2140.00


## 3-Parsing documents containing forms in tiff format

All the step are same as above only difference is change mime_type as 'image/tiff'

#### Create a function parse_form_tiff that take project_id and uri as an input and extract text information from the form

In [13]:
def parse_form_tiff(project_id,input_uri):

    client = documentai.DocumentUnderstandingServiceClient()
    gcs_source = documentai.types.GcsSource(uri=input_uri)
    input_config = documentai.types.InputConfig(
        gcs_source=gcs_source, mime_type='image/tiff')

    key_value_pair_hints = [
        documentai.types.KeyValuePairHint(key='Emergency Contact',
                                          value_types=['NAME']),
        documentai.types.KeyValuePairHint(
            key='Referred By')
    ]

    # Setting enabled=True enables form extraction
    form_extraction_params = documentai.types.FormExtractionParams(
        enabled=True, key_value_pair_hints=key_value_pair_hints)

    # Location can be 'us' or 'eu'
    parent = 'projects/{}/locations/us'.format(project_id)
    request = documentai.types.ProcessDocumentRequest(
        parent=parent,
        input_config=input_config,
        form_extraction_params=form_extraction_params)

    document = client.process_document(request=request)
    
    def _get_text(el):
        response = ''
        for segment in el.text_anchor.text_segments:
            start_index = segment.start_index
            end_index = segment.end_index
            response += document.text[start_index:end_index]
        return response

    for page in document.pages:
        print('Page number: {}'.format(page.page_number))
        for form_field in page.form_fields:
            print('Field Name: {}\tConfidence: {}'.format(_get_text(form_field.field_name),form_field.field_name.confidence))
            print('Field Value: {}\tConfidence: {}'.format(_get_text(form_field.field_value),form_field.field_value.confidence))

#### Call the function and print the response

In [14]:
project_id ='my-project-12195'
input_uri ='gs://training-12195/loan-form.tiff'
parse_form_tiff(project_id,input_uri)

Page number: 1
Field Name: Agreement Number:
	Confidence: 0.999992311000824
Field Value: 0123456789
	Confidence: 0.999992311000824
Field Name: Married 	Confidence: 0.9999892711639404
Field Value: D
	Confidence: 0.9999892711639404
Field Name: Other 	Confidence: 0.9999827146530151
Field Value: O
	Confidence: 0.9999827146530151
Field Name: Address:
	Confidence: 0.9999821186065674
Field Value: 100 Franklin Street, Mountain View, CA, 94035
	Confidence: 0.9999821186065674
Field Name: Address:
	Confidence: 0.9999768137931824
Field Value: 500 Castro Street, Mountain View, CA 94035
	Confidence: 0.9999768137931824
Field Name: Agreement date:
	Confidence: 0.9999767541885376
Field Value: 01/01/2020
	Confidence: 0.9999767541885376
Field Name: Phone number: 	Confidence: 0.9999765753746033
Field Value: 650-987-0934
	Confidence: 0.9999765753746033
Field Name: as the loan amount at
	Confidence: 0.999972403049469
Field Value: 6.0
	Confidence: 0.999972403049469
Field Name: Name:
	Confidence: 0.9999653100

## 4-Parsing documents containing forms in GIF format

All the step are same as above only difference is change mime_type as 'image/gif'

#### Create a function parse_form_gif that take project_id and uri as an input and extract text information from the form

In [15]:
def parse_form_gif(project_id,input_uri):

    client = documentai.DocumentUnderstandingServiceClient()
    gcs_source = documentai.types.GcsSource(uri=input_uri)
    input_config = documentai.types.InputConfig(
        gcs_source=gcs_source, mime_type='image/gif')

    key_value_pair_hints = [
        documentai.types.KeyValuePairHint(key='Emergency Contact',
                                          value_types=['NAME']),
        documentai.types.KeyValuePairHint(
            key='Referred By')
    ]

    # Setting enabled=True enables form extraction
    form_extraction_params = documentai.types.FormExtractionParams(
        enabled=True, key_value_pair_hints=key_value_pair_hints)

    # Location can be 'us' or 'eu'
    parent = 'projects/{}/locations/us'.format(project_id)
    request = documentai.types.ProcessDocumentRequest(
        parent=parent,
        input_config=input_config,
        form_extraction_params=form_extraction_params)

    document = client.process_document(request=request)
    
    def _get_text(el):
        response = ''
        for segment in el.text_anchor.text_segments:
            start_index = segment.start_index
            end_index = segment.end_index
            response += document.text[start_index:end_index]
        return response

    for page in document.pages:
        print('Page number: {}'.format(page.page_number))
        for form_field in page.form_fields:
            print('Field Name: {}\tConfidence: {}'.format(_get_text(form_field.field_name),form_field.field_name.confidence))
            print('Field Value: {}\tConfidence: {}'.format(_get_text(form_field.field_value),form_field.field_value.confidence))

#### Call the function and print the response

In [16]:
project_id ='my-project-12195'
input_uri ='gs://training-12195/loan-form.gif'
parse_form_gif(project_id,input_uri)

Page number: 1
Field Name: Agreement Number:
	Confidence: 0.9999967813491821
Field Value: 0123456789
	Confidence: 0.9999967813491821
Field Name: Address:
	Confidence: 0.9999939203262329
Field Value: 100 Franklin Street, Mountain View, CA, 94035
	Confidence: 0.9999939203262329
Field Name: Address:
	Confidence: 0.9999918937683105
Field Value: 500 Castro Street, Mountain View, CA 94035
	Confidence: 0.9999918937683105
Field Name: Phone number:
	Confidence: 0.9999865889549255
Field Value: 1-800-843-8623
	Confidence: 0.9999865889549255
Field Name: Agreement date:
	Confidence: 0.999984085559845
Field Value: 01/01/2020
	Confidence: 0.999984085559845
Field Name: Married 	Confidence: 0.9999793171882629
Field Value: o
	Confidence: 0.9999793171882629
Field Name: Other
	Confidence: 0.9999734163284302
Field Value: 	Confidence: 0.9999734163284302
Field Name: Name;
	Confidence: 0.999969482421875
Field Value: Arjun Patel
	Confidence: 0.999969482421875
Field Name: per month for
	Confidence: 0.9999612569